In [2]:
!pip install apache-beam
#pip install apache-beam[interactive]

^C



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for apache-beam from https://files.pythonhosted.org/packages/fd/8b/fd13e1d2898bb2823a8da680ef9ceae590b7774c96c4bf676692e4e7e757/apache_beam-2.54.0-cp310-cp310-win_amd64.whl.metadata
  Using cached apache_beam-2.54.0-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
  Using cached crcmod-1.7.tar.gz (89 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for orjson<4,>=3.9.7 from https://files.pythonhosted.org/packages/8f/6f/c6654219ad867ecebf85efbde6f6190634e9a6072246e63e2f7b60a1d717/orjson-3.9.15-cp310-none-win_amd64.whl.metadata
  Using cached orjson-3.9.15-cp310-none-win_amd64.whl.metadata (50 kB)
  Using cached dill-0.3.1.1.tar.gz (151 kB)
  I

In [ ]:
mkdir -p data
gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/

In [ ]:
import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

def takeSecond(elem):
    return elem[1]

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z]+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'sort descending' >> beam.combiners.Top.Of(20,key=takeSecond,reverse=False)
      | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

In [2]:
import apache_beam as beam
import re
import requests


inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'


def get_data():
  url = "https://imdb-top-100-movies.p.rapidapi.com/"
  headers = {
	"X-RapidAPI-Key": "11e969206dmsh2dc567692521a17p1aff25jsnf8d1e65d23f9",
	"X-RapidAPI-Host": "imdb-top-100-movies.p.rapidapi.com"}

  response = requests.request("GET", url, headers=headers)
  return list(response.json())


class json_to_txt(beam.DoFn):
  def __init__(self):
    pass

  def process(self, movie_json):
    values_list = list(movie_json.values())
    list_output = list(map(str, values_list))
    movie_str = '|'.join(list_output)
    yield movie_str


# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read Data From API' >> beam.Create(get_data())
      | 'Convert json to CSV' >> beam.ParDo(json_to_txt())
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )


In [ ]:
data =get_data()
columns = ['rank', 'title', 'thumbnail', 'rating', 'id', 'year', 'image', 'description', 'trailer','imdbid']

In [ ]:
columns = ['rank', 'title', 'thumbnail', 'rating', 'id', 'year', 'image', 'description', 'trailer','imdbid']

for a in data[0:1]:
  values_list = list(a.values())
  list_output = list(map(str, values_list))
  #print(list_output)
  print('|'.join(list_output))

1|The Shawshank Redemption|https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY67_CR0,0,45,67_AL_.jpg|9.3|top1|1994|https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX380_CR0,1,380,562_.jpg|Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.|https://www.youtube.com/embed/NmzuHjWmXOc|['Drama']|['Frank Darabont']|['Stephen King (based on the short novel "Rita Hayworth and the Shawshank Redemption" by)', 'Frank Darabont (screenplay by)']|tt0111161
